In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import PIL.Image
import networkx as nx
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import string

import sys
path=r'C:\Users\31649\Documents\genome analysis\genome_topology\functions'
sys.path.append(path)

from genome_topology import open_pdb
from genome_topology import select_chrom
from genome_topology import geom_distance
from genome_topology import fractal_dimension
from genome_topology import get_matrix
from genome_topology import normalize_psc

## Functions

In [2]:
def SplitChromosomesIntoSegments (coordinates, n_segments):
    length_coords= len(coordinates)
    length_segments = length_coords//n_segments
    rest=np.mod(length_coords,n_segments)
    segments = []

    for i in range(0, length_coords - rest, length_segments):
        segments.append(coordinates[i:i + length_segments])

    return segments

def write_topology_matrix(matrix, pathfile, namefile = 'top_matrix' ,format_output = 'feather'):
     
    size = mat.shape[0]
    mat_zeros = mat - np.ones((size, size))
    mat_upper = np.triu(mat_zeros)
    
    #get nonzero indexes for non-series matrix elements
    non_zero_indexes = np.nonzero(mat_upper)
    non_zero_indexes = np.transpose(non_zero_indexes)
    
    #find non zero (non-series) matrix elements
    mat_values = np.zeros(len(non_zero_indexes))
    mat_values = mat[non_zero_indexes[:,0], non_zero_indexes[:,1]]
    
    #save sparse representation in dataframe
    sparse_matrix = {'Index1': non_zero_indexes[:,0], 'Index2': non_zero_indexes[:,1], 'Values': mat_values}
    sparse_matrix = pd.DataFrame(sparse_matrix)
    
    #Print dataframe to file
    
    if (format_output == 'csv'):
        sparse_matrix.to_csv('{}/{}.csv'.format(pathfile, namefile))
    elif (format_output == 'feather'):
        sparse_matrix.to_feather('{}/{}.feather'.format(pathfile, namefile))
        
    return 'matrix printed to file'

def contact_indexes(pos1, pos2):
    index = [pos1, pos2]
    index=np.array(index)
    index= np.transpose(index)
    return index

# Split the chromosomes into segments to produce local matrices

##  Load data
Pick the resolution 

In [12]:
resolution = '40'
path = 'data/zoomify processed'
samples = ['Control1', 'Control2', 'Treated1', 'Treated2']
control1 = pd.DataFrame()
control2 = pd.DataFrame()
treated1 = pd.DataFrame()
treated2 = pd.DataFrame()

frames = [None]*4

for ind, sample in enumerate(samples):
    df =  pd.read_csv(f'{path}/{sample}{resolution}', sep = '\t', header = None, 
                      names = ['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2', 'count', 'balanced'],
                     dtype={'chrom1': 'str'})
    
    
    frames[ind] = df.dropna().reset_index(drop=True)
    sample_col = [sample]* len(frames[ind])
    frames[ind]['Sample'] = sample_col
    
    
control1 = frames[0]
control2 = frames[1]
treated1 = frames[2]
treated2 = frames[3]

## Filter data by their number of counts
 - Decide the quantile for data filtering 
 - Decide the type of counts you want to filter by (count/balanced count)

In [13]:
resolution

'40'

In [14]:
threshold_quantile = 0.99
count = 'count'

frames = [control1, control2, treated1, treated2]
contacts = pd.DataFrame()

total_data = list(control1[count]) + list(control2[count]) + list(treated1[count]) + list(treated2[count])
data = {'counts': total_data}
data = pd.DataFrame(data)
threshold = data['counts'].quantile(threshold_quantile)

for frame in frames:
    frame = frame[frame[count]>= threshold]
    frame = pd.DataFrame(frame)
    frames = [contacts, frame]
    contacts = pd.concat(frames)

    
new_index = np.linspace(1, len(contacts), len(contacts), dtype = int)
contacts['Index'] = new_index
contacts = contacts.set_index('Index')

In [15]:
contacts

,chrom1,start1,end1,chrom2,start2,end2,count,balanced,Sample
Index,,,,,,,,,
1,1,920000,960000,1,920000,960000,97,0.171960,Control1
2,1,920000,960000,1,1160000,1200000,34,0.071204,Control1
3,1,1120000,1160000,1,1120000,1160000,94,0.152657,Control1
4,1,1120000,1160000,1,1160000,1200000,77,0.154341,Control1
5,1,1120000,1160000,1,1200000,1240000,58,0.109523,Control1
...,...,...,...,...,...,...,...,...,...
1253027,X,154800000,154840000,X,154800000,154840000,80,0.153555,Treated2
1253028,X,154800000,154840000,X,154840000,154880000,46,0.091134,Treated2
1253029,X,154840000,154880000,X,154840000,154880000,54,0.110424,Treated2


In [16]:
chromosomes = control1['chrom1'].unique()


In [28]:
chromosomes = ['22']

## Split chromosomes in segments and produce topology matrices for the segments
Pick the path for saving the matrices

In [17]:
save_matrix = True

In [18]:
n_segments = 4
path_matrices = f'results counts/matrices/local matrices/{resolution}/{n_segments} segments'


for sample in samples:
    print(sample)
    contacts_sample = contacts[contacts['Sample'] == sample]
    for chrom in chromosomes:
        print(f'Chromosome {chrom}')
        contacts_chr= contacts_sample[(contacts_sample['chrom1']== chrom) & (contacts_sample['chrom2']==chrom)]
        
        length1 = (contacts_chr['end1'] -  contacts_chr['start1'])//2
        contacts_chr['position1']=  pd.Series(contacts_chr['start1'] + length1)
    
        length2 = (contacts_chr['end2'] -  contacts_chr['start2'])//2
        contacts_chr['position2']=  pd.Series(contacts_chr['start2'] + length2)
        
        maxim_coord = contacts_chr['end1'].max() 
        bins = np.linspace(0, maxim_coord, n_segments + 1,  dtype= int)
        
        segments = []

        for t in range(len(bins)-1):
            contacts_pos1 = contacts_chr[(contacts_chr['position1']> bins[t]) & (contacts_chr['position1']< bins[t + 1])]
            contacts_pos2 = contacts_pos1[(contacts_pos1['position2']> bins[t]) & (contacts_pos1['position2']< bins[t + 1])]
    
            index = contact_indexes(contacts_pos2['position1'], contacts_pos2['position2'])
            segments.append(index)
            
        for j in range(n_segments):
            print(j)
            N_contacts= len(segments[j])
            mat, psc = get_matrix(segments[j], 'segm')
            if save_matrix:
                write_topology_matrix(mat, path_matrices , namefile = f'{chrom}_{sample}_{j}')
            #plt.figure()
            #plt.imshow(mat)
        
        
        
        

Control1
Chromosome 1
0
1
2
3
Chromosome 2
0
1
2
3
Chromosome 3
0
1
2
3
Chromosome 4
0
1
2
3
Chromosome 5
0
1
2
3
Chromosome 6
0
1
2
3
Chromosome 7
0
1
2
3
Chromosome 8
0
1
2
3
Chromosome 9
0
1
2
3
Chromosome 10
0
1
2
3
Chromosome 11
0
1
2
3
Chromosome 12
0
1
2
3
Chromosome 13
0
1
2
3
Chromosome 14
0
1
2
3
Chromosome 15
0
1
2
3
Chromosome 16
0
1
2
3
Chromosome 17
0
1
2
3
Chromosome 18
0
1
2
3
Chromosome 19
0
1
2
3
Chromosome 20
0
1
2
3
Chromosome 21
0
1
2
3
Chromosome 22
0
1
2
3
Chromosome X
0
1
2
3
Control2
Chromosome 1
0
1
2
3
Chromosome 2
0
1
2
3
Chromosome 3
0
1
2
3
Chromosome 4
0
1
2
3
Chromosome 5
0
1
2
3
Chromosome 6
0
1
2
3
Chromosome 7
0
1
2
3
Chromosome 8
0
1
2
3
Chromosome 9
0
1
2
3
Chromosome 10
0
1
2
3
Chromosome 11
0
1
2
3
Chromosome 12
0
1
2
3
Chromosome 13
0
1
2
3
Chromosome 14
0
1
2
3
Chromosome 15
0
1
2
3
Chromosome 16
0
1
2
3
Chromosome 17
0
1
2
3
Chromosome 18
0
1
2
3
Chromosome 19
0
1
2
3
Chromosome 20
0
1
2
3
Chromosome 21
0
1
2
3
Chromosome 22
0
1
2
3
Chromosome 

In [11]:
contacts_chr

,chrom1,start1,end1,chrom2,start2,end2,count,balanced,Sample,position1,position2
Index,,,,,,,,,,,
1637356,X,2780000,2800000,X,2780000,2800000,26,0.089296,Treated2,2790000,2790000
1637357,X,2900000,2920000,X,2900000,2920000,45,0.098711,Treated2,2910000,2910000
1637358,X,2920000,2940000,X,2920000,2940000,22,0.075060,Treated2,2930000,2930000
1637359,X,2920000,2940000,X,2940000,2960000,24,0.068358,Treated2,2930000,2950000
1637360,X,2940000,2960000,X,2940000,2960000,37,0.087977,Treated2,2950000,2950000
...,...,...,...,...,...,...,...,...,...,...,...
1646356,X,154860000,154880000,X,154880000,154900000,22,0.070930,Treated2,154870000,154890000
1646357,X,154880000,154900000,X,154880000,154900000,31,0.069182,Treated2,154890000,154890000
1646358,X,154880000,154900000,X,154900000,154920000,21,0.046109,Treated2,154890000,154910000


In [19]:
bins

array([       0, 12800000, 25600000, 38400000, 51200000])

In [20]:
index

array([[38480000, 38480000],
       [38480000, 38640000],
       [38480000, 38800000],
       ...,
       [50960000, 50960000],
       [50960000, 51120000],
       [51120000, 51120000]], dtype=int64)

In [21]:
segments

[array([], shape=(0, 2), dtype=int64),
 array([[17360000, 17360000],
        [17360000, 17520000],
        [17360000, 17680000],
        [17360000, 17840000],
        [17360000, 18000000],
        [17360000, 18160000],
        [17360000, 18320000],
        [17360000, 18480000],
        [17360000, 19600000],
        [17520000, 17520000],
        [17520000, 17680000],
        [17520000, 17840000],
        [17520000, 18000000],
        [17520000, 18160000],
        [17520000, 18320000],
        [17520000, 18480000],
        [17520000, 18640000],
        [17680000, 17680000],
        [17680000, 17840000],
        [17680000, 18000000],
        [17680000, 18160000],
        [17680000, 18320000],
        [17680000, 18480000],
        [17680000, 18640000],
        [17680000, 18960000],
        [17840000, 17840000],
        [17840000, 18000000],
        [17840000, 18160000],
        [17840000, 18320000],
        [17840000, 18480000],
        [17840000, 18640000],
        [17840000, 19440000],
 